In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('weather.csv')
data['date']=pd.to_datetime(data['date'])

FileNotFoundError: [Errno 2] File b'weather.csv' does not exist: b'weather.csv'

In [2]:
data = pd.read_csv('weather.csv')
data = data.loc[:,['station_nbr','date','codesum']]
data['date']=pd.to_datetime(data['date'])

# Add variables
data['is_weekend'] = data['date'].dt.dayofweek.apply(lambda x: int(x in [5,6]))
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [3]:
# Add is_holiday
from datetime import datetime
date_format = "%Y-%m-%d"

holiday_list = ["2012-01-01", "2012-01-16", "2012-02-14", "2012-02-20", "2012-04-08",
                "2012-05-13", "2012-05-28", "2012-06-17", "2012-07-04", "2012-09-03",
                "2012-10-08", "2012-10-31", "2012-11-11", "2012-11-20", "2012-11-21",
                "2012-11-22", "2012-11-23", "2012-11-24", "2012-11-25", "2012-11-26",
                "2012-12-24", "2012-12-25", "2012-12-31", "2013-01-01", "2013-01-21",
                "2013-02-14", "2013-02-18", "2013-05-31", "2013-05-12", "2013-05-27",
                "2013-06-16", "2013-07-04", "2013-09-02", "2013-10-14", "2013-10-31",
                "2013-11-11", "2013-11-26", "2013-11-27", "2013-11-28", "2013-11-29",
                "2013-11-30", "2013-12-01", "2013-12-02", "2013-12-24", "2013-12-25",
                "2013-12-31", "2014-01-01", "2014-01-20", "2014-02-14", "2014-02-17",
                "2014-04-20", "2014-05-11", "2014-05-26", "2014-06-15", "2014-07-04",
                "2014-09-01", "2014-10-13", "2014-10-31"]

holidays = [datetime.strptime(holiday, date_format) for holiday in holiday_list]

data['is_holiday'] = 0

for i in range(0, data.shape[0]):
    if data["date"][i] in holidays:
        data.is_holiday.iloc[i] = 1

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
def text_process(txt):
    
    #remove punctuation
    sent = re.sub('[^a-zA-Z]', ' ', txt)
    sent = re.sub(r"\s+", "", sent)
    words = []
    for i in range(0, len(sent), 2):
        
        words.append(sent[i:i+2]) if sent[i:i+2] not in words else None
    
    return words

In [5]:
cv = CountVectorizer(analyzer = text_process, max_features=33)
codesum = cv.fit_transform(data.codesum)
condesum_df = pd.DataFrame(codesum.toarray(),index=data.index,columns=sorted(cv.vocabulary_.keys()))
data = data.drop('codesum', axis = 1)
data = pd.concat([condesum_df, data], axis = 1)
del codesum

In [6]:
train = pd.read_csv('train.csv')
train.date = pd.to_datetime(train.date)
key = pd.read_csv('key.csv')
data = pd.merge(data, key, how = 'left', on = 'station_nbr')
training = pd.merge(data,train,how = 'right', on = ['date', 'store_nbr'])

In [8]:
test = pd.read_csv('test.csv')
test.date = pd.to_datetime(test.date)
testing = pd.merge(data,test,how = 'right', on = ['date', 'store_nbr'])
ytrain = training['units']
xtrain = training.drop(['station_nbr','date', 'units'], axis = 1)
xtest = testing.drop(['station_nbr', 'date'], axis = 1)

In [19]:
for store_nbr in range(1, 46):
    print(store_nbr, "/ 45")
    for item_nbr in range(1, 112):
        # extract one specific comb of store nbr and item nbr
        temp_data = training[(training["store_nbr"] == store_nbr) & (training["item_nbr"] == item_nbr)]
        
        # check if all units in the temp data are zeros
        if np.mean(temp_data["units"] == 0) == 1.0:
            
            # delete all indices in the temp data from the whole data
            training = training.drop(axis = 0, index = temp_data.index)

1 / 45
2 / 45
3 / 45
4 / 45
5 / 45
6 / 45
7 / 45
8 / 45
9 / 45
10 / 45
11 / 45
12 / 45
13 / 45
14 / 45
15 / 45
16 / 45
17 / 45
18 / 45
19 / 45
20 / 45
21 / 45
22 / 45
23 / 45
24 / 45
25 / 45
26 / 45
27 / 45
28 / 45
29 / 45
30 / 45
31 / 45
32 / 45
33 / 45
34 / 45
35 / 45
36 / 45
37 / 45
38 / 45
39 / 45
40 / 45
41 / 45
42 / 45
43 / 45
44 / 45
45 / 45


In [23]:
training.to_csv("training_data.csv", index = False)

In [24]:
deleted_list = []
saved_list = []
for store_nbr in range(1, 46):
    for item_nbr in range(1, 112):
        # extract one specific comb of store nbr and item nbr
        temp_data = training[(training["store_nbr"] == store_nbr) & (training["item_nbr"] == item_nbr)]
        
        if temp_data.shape[0] == 0:
            deleted_list.append((store_nbr, item_nbr))
        else:
            saved_list.append((store_nbr, item_nbr))

In [25]:
import pickle
# save deleted_list as a file
with open("deleted_list.txt", "wb") as dl:
    pickle.dump(deleted_list, dl)

In [26]:
# save saved_list as a file
with open("saved_list.txt", "wb") as sl:
    pickle.dump(saved_list, sl)

In [ ]:
# Run the following code to load lists 
with open("deleted_list.txt", "rb") as dl:
    deleted_list = pickle.load(dl)

with open("saved_list.txt", "rb") as sl:
    saved_list = pickle.load(sl)

In [27]:
from scipy.sparse import csr_matrix
xtrain1 = csr_matrix(xtrain.values,shape = xtrain.shape, dtype=float)
xtest1 = csr_matrix(xtest.values,shape = xtest.shape,dtype=float)

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [12]:
gb = GradientBoostingRegressor().fit(xtrain, ytrain)
ypred = gb.predict(xtest)

In [13]:
result = pd.DataFrame(ypred)

In [ ]:
result.to_excel("output.xlsx")